In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [0]:
spark = SparkSession.builder.appName("Ecomdatapipeline").getOrCreate()


#Read the necessary Silver Tables

silver_sellers = spark.read.format("delta").load("/mnt/delta/tables/ecom2/silver/sellers")
silver_country = spark.read.format("delta").load("/mnt/delta/tables/ecom2/silver/country")
silver_users = spark.read.format("delta").load("/mnt/delta/tables/ecom2/silver/users")
silver_buyers = spark.read.format("delta").load("/mnt/delta/tables/ecom2/silver/buyers")

In [0]:
# Performing Join operations

comprehensive_user_table = silver_users\
    .join(silver_country,silver_users.country == silver_country.country,"outer")\
    .join(silver_buyers,silver_users.country == silver_buyers.country,"outer")\
    .join(silver_sellers,silver_users.country == silver_sellers.country,"outer")




In [0]:
#Select and alias columns from each dataframe to ensure uniqueness

comprehensive_user_table = comprehensive_user_table.select(
    silver_users["country"].alias("Country"),
    silver_users["productsSold"].alias("Users_productsSold"),
    silver_users["productsWished"].alias("Users_productsWished"),
    silver_users["account_age_years"].alias("Users_account_age_years"),
    silver_users["account_age_group"].alias("Users_account_age_group"),
    silver_users["hasAnyApp"].alias("Users_hasAnyApp"),
    silver_users["socialNbFollowers"].alias("Users_socialNbFollowers"),
    silver_users["flag_long_title"].alias("Users_flag_long_title"),

    silver_country["sellers"].alias("Country_Sellers"),
    silver_country["topsellers"].alias("Country_topsellers"),
    silver_country["femalesellers"].alias("Country_femalesellers"),
    silver_country["malesellers"].alias("Country_malesellers"),
    silver_country["topfemalesellers"].alias("Country_topfemalesellers"),
    silver_country["topmalesellers"].alias("Country_topmalesellers"),



    silver_buyers["buyers"].alias("Buyers_Total"),
    silver_buyers["topbuyers"].alias("Buyers_Top"),
    silver_buyers["femalebuyers"].alias("Buyers_Female"),
    silver_buyers["malebuyers"].alias("Buyers_Male"),
    silver_buyers["topfemalebuyers"].alias("Buyers_TopFemale"),
    silver_buyers["topmalebuyers"].alias("Buyers_TopMale"),


    silver_sellers["nbsellers"].alias("Sellers_Total"),
    silver_sellers["sex"].alias("Sellers_Sex"),
    silver_sellers["meanproductssold"].alias("Sellers_meanproductssold"),
    silver_sellers["meanproductslisted"].alias("Sellers_meanproductslisted")

)

In [0]:
comprehensive_user_table.write.format("delta").mode("overwrite").option("mergeSchema", "true").save("/mnt/delta/tables/ecom2/gold/ecom_one_big_table")

In [0]:
%sql
CREATE OR REPLACE TABLE ecom_one_big_table
USING DELTA
AS 
SELECT *
from delta.`/mnt/delta/tables/ecom2/gold/ecom_one_big_table`;

num_affected_rows,num_inserted_rows
